<a href="https://colab.research.google.com/github/avkornaev/Cells_counter/blob/main/Week_12/Hands_on_CLIPSeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero-Shot Image Segmentation with CLIPSeg

**Implementation** Note:  
This notebook was developed using methodologies suggested by  
the DeepSeek-V3 language model (DeepSeek, 2024).

In [ ]:
# Install required libraries
!pip install -q transformers torch pillow

In [ ]:
import requests
import torch
from torch.nn.functional import interpolate

from PIL import Image

from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import gridspec

from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np

from transformers import CLIPSegProcessor, CLIPSegForImageSegmentation

In [ ]:
#@title Image Input Options {run: "auto"}
input_method = "URL" #@param ["URL", "Upload"]

if input_method == "URL":
    # Use example image or paste your own URL
    url = "https://images.unsplash.com/photo-1573865526739-10659fec78a5"  # Cat image
    image = Image.open(requests.get(url, stream=True).raw)
else:
    # Upload custom image
    from google.colab import files
    uploaded = files.upload()
    file_name = next(iter(uploaded))
    image = Image.open(file_name)

# Display the image
plt.imshow(np.array(image))
plt.axis('off')
plt.show()

In [ ]:
# Load model
model = CLIPSegForImageSegmentation.from_pretrained("CIDAS/clipseg-rd64-refined")
processor = CLIPSegProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")


# Define text prompts (multiple supported)
texts = ["a cat", "eyes", "a cat's paws", "a cat's whiskers",
         "a cat's tail", "a table"]
# Process inputs
inputs = processor(text=texts, images=[image]*len(texts), return_tensors="pt", padding=True)

# Predict
with torch.no_grad():
    outputs = model(**inputs)

# Visualize masks
fig, axes = plt.subplots(1, len(texts), figsize=(15,5))
for idx, (text, mask) in enumerate(zip(texts, outputs.logits)):
    axes[idx].imshow(mask.sigmoid().numpy(), cmap='viridis')
    axes[idx].set_title(text)
    axes[idx].axis('off')
plt.show()

In [ ]:
for idx, (text, mask) in enumerate(zip(texts, outputs.logits)):
    mask_norm = mask.sigmoid().numpy()
    mean_confidence = mask_norm.mean()
    print(mean_confidence)

In [ ]:
#title Enhanced Segmentation Visualization with Probability Sidebars

# Create figure
fig = plt.figure(figsize=(20, 5 * len(texts)))  # Adjust height based on number of texts

for idx, (text, mask) in enumerate(zip(texts, outputs.logits)):
    # Create grid layout for this segmentation
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1], wspace=0.05)

    # Main image with mask overlay
    ax0 = plt.subplot(gs[0])
    img_array = np.array(image)

    # Resize mask to match image
    mask_resized = interpolate(
        mask.unsqueeze(0).unsqueeze(0),
        size=img_array.shape[:2],
        mode='bilinear'
    ).squeeze().numpy()
    mask_norm = 1 / (1 + np.exp(-mask_resized))

    # Display
    ax0.imshow(img_array)
    overlay = ax0.imshow(mask_norm, cmap='viridis', alpha=0.5)
    ax0.set_title(f"'{text}' Segmentation", fontsize=12, pad=10)
    ax0.axis('off')

    # Add colorbar sidebar
    ax1 = plt.subplot(gs[1])
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="20%", pad=0.1)

    # Create histogram of probabilities
    hist_data = mask_norm.flatten()
    hist_data = hist_data[hist_data > 0.1]  # Filter very low probabilities

    ax1.hist(hist_data, bins=50, orientation='horizontal', color='green')
    ax1.set_xlabel('Pixel Count')
    ax1.set_ylabel('Mask Probability')
    ax1.set_title('Probability Distribution', fontsize=10)
    ax1.grid(True, alpha=0.3)

    # Add colorbar
    plt.colorbar(overlay, cax=cax, label='Mask Confidence')

plt.tight_layout()
plt.show()